In [22]:
import matplotlib.pyplot as plt
from matplotlib import colors, cm
import numpy as np
import pandas as pd
import logging as log
from stompy.grid import unstructured_grid
import glob
import re
from stompy import memoize
from stompy.plot import plot_wkb
import six
import stompy.plot.cmap as scmap

import os
from stompy import utils
import xarray as xr
%matplotlib notebook


In [10]:
import postprocess_v00 as post
six.moves.reload_module(post)

<module 'postprocess_v00' from '/home/rusty/src/microplastic_sfbay/postprocess/postprocess_v00.py'>

In [11]:
grid_fn="/opt2/sfb_ocean/suntans/runs/merged_018_20171227/ptm_average.nc_0000.nc"  
grid=post.grid_from_ptm_hydro(grid_fn)
M=grid.smooth_matrix(f=0.5,dx='grid',A='grid',V='grid',K='scaled')

INFO:utils:59307/99089


In [12]:
grid_poly=grid.boundary_polygon()

INFO:join_features:0 open strings, 25 simple polygons
INFO:join_features:Building index
INFO:join_features:done building index
INFO:join_features:Examining largest poly left with area=38109911619.298943, 24 potential interiors


In [13]:
def smooth(c): 
    for _ in range(20):
        c=M.dot(c)
    return c

In [14]:
#cmap=scmap.load_gradient('001-fire.cpt',reverse=True)
cmap=cm.CMRmap_r
cmap=scmap.cmap_clip(cmap,0.03,1.0)

In [15]:
class BayConcFigure(object):
    figsize=(8.4,8)
    ax=None
    fig=None
    vmin=1e-5
    vmax=10.0
    zoom=(517521., 609000., 4139744., 4230000.)
    cmap=cmap
    cax_loc=[0.7,0.25,0.03,0.35]
    txt_loc=[0.65,0.7] # in ax coords
    cbar_label="Particles/m$^2$"
    cbar_args={} # don't modify - replace.
    draw_boundary=True
    fontsize=14
    extra_text=[]
    num=None
    def __init__(self,ds,conc=None,**kw):
        utils.set_keywords(self,kw)
        self.ds=ds
        if conc is None:
            conc=self.ds['conc'].values

        if self.fig is None:
            self.fig=plt.figure(figsize=self.figsize,num=self.num)
        if self.ax is None:
            self.ax=self.fig.add_subplot(1,1,1)
        
        self.ccoll=grid.plot_cells(values=conc.clip(self.vmin,self.vmax),
                                   cmap=self.cmap,norm=colors.LogNorm(vmin=self.vmin,vmax=self.vmax),
                                   edgecolor='face',lw=0.4,ax=self.ax)
        if self.draw_boundary:
            self.boundary=plot_wkb.plot_wkb(grid_poly,ax=self.ax,ec='k',lw=0.5,fc='none')
        if self.cax_loc is not None:
            self.cax=self.fig.add_axes(self.cax_loc) # Need refactor
            plt.colorbar(self.ccoll,cax=self.cax,label=self.cbar_label,extend='both',
                         **self.cbar_args)
        self.ax.axis('equal')
        self.ax.axis(self.zoom)
        self.ax.xaxis.set_visible(0)
        self.ax.yaxis.set_visible(0)
        self.fig.subplots_adjust(left=0.01,right=0.99,top=0.99,bottom=0.01)
        
        self.add_labels()
    def __del__(self):
        try:
            self.ds.close()
        except AttributeError:
            pass
    def add_labels(self):
        texts=[self.behavior_label()]
        texts.append(self.average_label())
        texts.append(self.date_label())
        texts+=self.extra_text
        self.ax.text(self.txt_loc[0],self.txt_loc[1],"\n".join(texts),
                     fontsize=self.fontsize,va='top',ha='left',transform=self.ax.transAxes)
    def behavior_label(self):
        # go from a list of groups to a label
        behaviors=[ post.PtmRun.group_to_src_behavior(group)[1]
                    for group in self.ds.ptm_groups.values ]
        behaviors.sort()
        assert behaviors[0]==behaviors[-1]
        behavior=behaviors[0]
        if behavior=='none':
            label='Passive'
        elif behavior.startswith('up'):
            w_mmps=float(behavior.replace('up',''))/1000.0
            label=f'Rise {w_mmps:.1f} mm/s'
        elif behavior.startswith('down'):
            w_mmps=float(behavior.replace('down',''))/1000.0
            label=f'Sink {w_mmps:.1f} mm/s'
        return label
    
    def date_label(self):
        return self.date_label_ext(self.ds)
    @classmethod
    def date_label_ext(cls,ds):
        def fmt_t(t): return utils.to_datetime(t).strftime('%Y-%m-%d')
        return f"{fmt_t(ds.time_start)} – {fmt_t(ds.time_end)}"
    
    def average_label(self):
        return self.average_label_ext(self.ds)
    @classmethod
    def average_label_ext(cls,ds):
        if np.all(ds.z_range.values==[0,0.5]):
            return "Near bed"
        elif np.all(ds.z_range.values==[-0.5,0.0]):
            return "Near surface"
        elif np.all(ds.z_range.values==[0,0]):
            return "Full depth"
        else:
            return "Vertical range: %.2f – %.2f"%(ds.z_range.values[0],
                                                  ds.z_range.values[1])

# Same idea but settings for coastal region.
# note that with the 15 day output, these get truncated
# and show little action in the ocean.
class CoastalConcFigure(BayConcFigure):
    figsize=(7,8.1)
    zoom=(345000., 613202., 4050000., 4230105.)
    cax_loc=[0.05,0.20,0.03,0.35]
    txt_loc=[0.05,0.13] # in ax coords
    

In [16]:
# This should be the only place where version specific strings
# appear.

def single_fns():
    fns=glob.glob('processed/201*_201*/[-0-9]*/particles*-v07std.nc')
    fns.sort()
    return fns

@memoize.memoize(lru=10)
def load_ptm_conc(ptm_conc):
    ds=xr.open_dataset(ptm_conc)
    conc=smooth(ds.conc.values)
    return ds,conc

@memoize.memoize(lru=10)
def load_combined(ptm_concA):
    ptm_concB=ptm_concA.replace('v07A','v07B')
    assert ptm_concB!=ptm_concA
    dsA=xr.open_dataset(ptm_concA)
    dsB=xr.open_dataset(ptm_concB)
    conc=smooth(dsA.conc.values+dsB.conc.values)
    # might need to shallow copy dsA and modify
    return dsA,conc

def combined_fns():
    fns=glob.glob('processed/201*_201*/[-0-9]*/particles*-v07A*.nc')
    fns.sort()
    return fns


In [20]:
# The combined figures
display=False
force=False

if not display: plt.ioff()

for ptm_conc in single_fns():
    ds=None
    conc=None
    for config in ['bay','coast']:
        img_fn=ptm_conc.replace('.nc',f'-{config}.png')
        
        if (not display) and os.path.exists(img_fn) and not force: 
            log.info(f"{img_fn}: exists")
            continue
        log.info(f"{img_fn}: processing")
        if ds is None:
            ds=xr.open_dataset(ptm_conc)
        if conc is None:
            conc=smooth(ds.conc.values)
        if config=='bay':
            bf=BayConcFigure(ds,conc=conc)
        elif config=='coast':
            bf=CoastalConcFigure(ds,conc=conc)
        else:
            raise Exception(f"Bad config {config}")
        if display:
            break
        bf.fig.savefig(img_fn,dpi=150)
        plt.close(bf.fig)
    
if not display: plt.ion()

INFO:root:processed/20170630_20170715/-0.0005/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20170630_20170715/-0.0005/particles-avg-15days-v07std-coast.png: exists
INFO:root:processed/20170630_20170715/-0.0005/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/20170630_20170715/-0.0005/particles-bed-15days-v07std-coast.png: exists
INFO:root:processed/20170630_20170715/-0.0005/particles-surf-15days-v07std-bay.png: exists
INFO:root:processed/20170630_20170715/-0.0005/particles-surf-15days-v07std-coast.png: exists
INFO:root:processed/20170630_20170715/-0.005/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20170630_20170715/-0.005/particles-avg-15days-v07std-coast.png: exists
INFO:root:processed/20170630_20170715/-0.005/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/20170630_20170715/-0.005/particles-bed-15days-v07std-coast.png: exists
INFO:root:processed/20170630_20170715/-0.005/particles-surf-15days-v07std-bay.png: exists
IN

INFO:root:processed/20170830_20170914/-0.005/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/20170830_20170914/-0.005/particles-bed-15days-v07std-coast.png: exists
INFO:root:processed/20170830_20170914/-0.005/particles-surf-15days-v07std-bay.png: exists
INFO:root:processed/20170830_20170914/-0.005/particles-surf-15days-v07std-coast.png: exists
INFO:root:processed/20170830_20170914/-0.05/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20170830_20170914/-0.05/particles-avg-15days-v07std-coast.png: exists
INFO:root:processed/20170830_20170914/-0.05/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/20170830_20170914/-0.05/particles-bed-15days-v07std-coast.png: exists
INFO:root:processed/20170830_20170914/-0.05/particles-surf-15days-v07std-bay.png: exists
INFO:root:processed/20170830_20170914/-0.05/particles-surf-15days-v07std-coast.png: exists
INFO:root:processed/20170830_20170914/0.0/particles-avg-15days-v07std-bay.png: exists
INFO:root:proces

INFO:root:processed/20171030_20171114/-0.05/particles-surf-15days-v07std-bay.png: exists
INFO:root:processed/20171030_20171114/-0.05/particles-surf-15days-v07std-coast.png: exists
INFO:root:processed/20171030_20171114/0.0/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20171030_20171114/0.0/particles-avg-15days-v07std-coast.png: exists
INFO:root:processed/20171030_20171114/0.0/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/20171030_20171114/0.0/particles-bed-15days-v07std-coast.png: exists
INFO:root:processed/20171030_20171114/0.0/particles-surf-15days-v07std-bay.png: exists
INFO:root:processed/20171030_20171114/0.0/particles-surf-15days-v07std-coast.png: exists
INFO:root:processed/20171030_20171114/0.0005/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20171030_20171114/0.0005/particles-avg-15days-v07std-coast.png: exists
INFO:root:processed/20171030_20171114/0.0005/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/2017103

INFO:root:processed/20171230_20180114/0.0005/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20171230_20180114/0.0005/particles-avg-15days-v07std-coast.png: exists
INFO:root:processed/20171230_20180114/0.0005/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/20171230_20180114/0.0005/particles-bed-15days-v07std-coast.png: exists
INFO:root:processed/20171230_20180114/0.0005/particles-surf-15days-v07std-bay.png: exists
INFO:root:processed/20171230_20180114/0.0005/particles-surf-15days-v07std-coast.png: exists
INFO:root:processed/20171230_20180114/0.005/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20171230_20180114/0.005/particles-avg-15days-v07std-coast.png: exists
INFO:root:processed/20171230_20180114/0.005/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/20171230_20180114/0.005/particles-bed-15days-v07std-coast.png: exists
INFO:root:processed/20171230_20180114/0.005/particles-surf-15days-v07std-bay.png: exists
INFO:root:pro

INFO:root:processed/20180302_20180317/0.005/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/20180302_20180317/0.005/particles-bed-15days-v07std-coast.png: exists
INFO:root:processed/20180302_20180317/0.005/particles-surf-15days-v07std-bay.png: exists
INFO:root:processed/20180302_20180317/0.005/particles-surf-15days-v07std-coast.png: exists
INFO:root:processed/20180302_20180317/0.05/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20180302_20180317/0.05/particles-avg-15days-v07std-coast.png: exists
INFO:root:processed/20180302_20180317/0.05/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/20180302_20180317/0.05/particles-bed-15days-v07std-coast.png: exists
INFO:root:processed/20180302_20180317/0.05/particles-surf-15days-v07std-bay.png: exists
INFO:root:processed/20180302_20180317/0.05/particles-surf-15days-v07std-coast.png: exists
INFO:root:processed/20180330_20180414/-0.0005/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20

INFO:root:processed/20180430_20180515/0.05/particles-surf-15days-v07std-bay.png: exists
INFO:root:processed/20180430_20180515/0.05/particles-surf-15days-v07std-coast.png: exists
INFO:root:processed/20180530_20180614/-0.0005/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20180530_20180614/-0.0005/particles-avg-15days-v07std-coast.png: exists
INFO:root:processed/20180530_20180614/-0.0005/particles-bed-15days-v07std-bay.png: exists
INFO:root:processed/20180530_20180614/-0.0005/particles-bed-15days-v07std-coast.png: exists
INFO:root:processed/20180530_20180614/-0.0005/particles-surf-15days-v07std-bay.png: exists
INFO:root:processed/20180530_20180614/-0.0005/particles-surf-15days-v07std-coast.png: exists
INFO:root:processed/20180530_20180614/-0.005/particles-avg-15days-v07std-bay.png: exists
INFO:root:processed/20180530_20180614/-0.005/particles-avg-15days-v07std-coast.png: exists
INFO:root:processed/20180530_20180614/-0.005/particles-bed-15days-v07std-bay.png: exists
INFO:

In [23]:
display=False
if not display: plt.ioff()

for ptm_concA in combined_fns():
    for config in ['bay','coast']:
        img_fn=re.sub('(v\d\d)A',r'\1AB',ptm_concA).replace('.nc',f'-{config}.png')
    
        if (not display) and os.path.exists(img_fn): 
            log.info(f"{img_fn}: exists")
            continue
        log.info(f"{img_fn}: processing")
        ds,conc=load_combined(ptm_concA)
        kw=dict(extra_text=['44 day cutoff'])
        if config=='bay':
            bf=BayConcFigure(ds,conc=conc,txt_loc=[0.65,0.75],**kw)
        elif config=='coast':
            bf=CoastalConcFigure(ds,conc=conc,**kw)
        else:
            raise Exception(f"Bad config {config}")
        if not display:
            bf.fig.savefig(img_fn,dpi=150)
            plt.close(bf.fig)
    if display:
        break
    
if not display: plt.ion()

INFO:root:processed/20170729_20170812/-0.0005/particles-avg-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20170729_20170812/-0.0005/particles-avg-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20170729_20170812/-0.0005/particles-avg-44days-v07ABstd-bay.png: processing
INFO:root:processed/20170729_20170812/-0.0005/particles-avg-44days-v07ABstd-coast.png: processing
INFO:root:processed/20170729_20170812/-0.0005/particles-bed-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20170729_20170812/-0.0005/particles-bed-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20170729_20170812/-0.0005/particles-bed-44days-v07ABstd-bay.png: processing
INFO:root:processed/20170729_20170812/-0.0005/particles-bed-44days-v07ABstd-coast.png: processing
INFO:root:processed/20170729_20170812/-0.0005/particles-surf-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20170729_20170812/-0.0005/particles-surf-44days-v07ABnofiber-coast.png: processing
INFO

INFO:root:processed/20170828_20170911/-0.0005/particles-avg-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20170828_20170911/-0.0005/particles-avg-44days-v07ABstd-bay.png: processing
INFO:root:processed/20170828_20170911/-0.0005/particles-avg-44days-v07ABstd-coast.png: processing
INFO:root:processed/20170828_20170911/-0.0005/particles-bed-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20170828_20170911/-0.0005/particles-bed-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20170828_20170911/-0.0005/particles-bed-44days-v07ABstd-bay.png: processing
INFO:root:processed/20170828_20170911/-0.0005/particles-bed-44days-v07ABstd-coast.png: processing
INFO:root:processed/20170828_20170911/-0.0005/particles-surf-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20170828_20170911/-0.0005/particles-surf-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20170828_20170911/-0.0005/particles-surf-44days-v07ABstd-bay.png: processing
INFO:ro

INFO:root:processed/20170928_20171012/-0.0005/particles-avg-44days-v07ABstd-bay.png: processing
INFO:root:processed/20170928_20171012/-0.0005/particles-avg-44days-v07ABstd-coast.png: processing
INFO:root:processed/20170928_20171012/-0.0005/particles-bed-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20170928_20171012/-0.0005/particles-bed-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20170928_20171012/-0.0005/particles-bed-44days-v07ABstd-bay.png: processing
INFO:root:processed/20170928_20171012/-0.0005/particles-bed-44days-v07ABstd-coast.png: processing
INFO:root:processed/20170928_20171012/-0.0005/particles-surf-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20170928_20171012/-0.0005/particles-surf-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20170928_20171012/-0.0005/particles-surf-44days-v07ABstd-bay.png: processing
INFO:root:processed/20170928_20171012/-0.0005/particles-surf-44days-v07ABstd-coast.png: processing
INFO:root:

INFO:root:processed/20171029_20171112/-0.0005/particles-avg-44days-v07ABstd-coast.png: processing
INFO:root:processed/20171029_20171112/-0.0005/particles-bed-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20171029_20171112/-0.0005/particles-bed-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20171029_20171112/-0.0005/particles-bed-44days-v07ABstd-bay.png: processing
INFO:root:processed/20171029_20171112/-0.0005/particles-bed-44days-v07ABstd-coast.png: processing
INFO:root:processed/20171029_20171112/-0.0005/particles-surf-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20171029_20171112/-0.0005/particles-surf-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20171029_20171112/-0.0005/particles-surf-44days-v07ABstd-bay.png: processing
INFO:root:processed/20171029_20171112/-0.0005/particles-surf-44days-v07ABstd-coast.png: processing
INFO:root:processed/20171029_20171112/-0.005/particles-avg-44days-v07ABnofiber-bay.png: processing
INFO:ro

INFO:root:processed/20171128_20171212/-0.0005/particles-bed-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20171128_20171212/-0.0005/particles-bed-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20171128_20171212/-0.0005/particles-bed-44days-v07ABstd-bay.png: processing
INFO:root:processed/20171128_20171212/-0.0005/particles-bed-44days-v07ABstd-coast.png: processing
INFO:root:processed/20171128_20171212/-0.0005/particles-surf-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20171128_20171212/-0.0005/particles-surf-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20171128_20171212/-0.0005/particles-surf-44days-v07ABstd-bay.png: processing
INFO:root:processed/20171128_20171212/-0.0005/particles-surf-44days-v07ABstd-coast.png: processing
INFO:root:processed/20171128_20171212/-0.005/particles-avg-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20171128_20171212/-0.005/particles-avg-44days-v07ABnofiber-coast.png: processing
INFO

...

INFO:root:processed/20180128_20180211/-0.0005/particles-bed-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180128_20180211/-0.0005/particles-bed-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180128_20180211/-0.0005/particles-surf-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180128_20180211/-0.0005/particles-surf-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180128_20180211/-0.0005/particles-surf-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180128_20180211/-0.0005/particles-surf-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180128_20180211/-0.005/particles-avg-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180128_20180211/-0.005/particles-avg-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180128_20180211/-0.005/particles-avg-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180128_20180211/-0.005/particles-avg-44days-v07ABstd-coast.png: processing
INFO:root:proc

INFO:root:processed/20180228_20180314/-0.0005/particles-bed-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180228_20180314/-0.0005/particles-surf-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180228_20180314/-0.0005/particles-surf-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180228_20180314/-0.0005/particles-surf-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180228_20180314/-0.0005/particles-surf-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180228_20180314/-0.005/particles-avg-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180228_20180314/-0.005/particles-avg-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180228_20180314/-0.005/particles-avg-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180228_20180314/-0.005/particles-avg-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180228_20180314/-0.005/particles-bed-44days-v07ABnofiber-bay.png: processing
INFO:root:p

INFO:root:processed/20180331_20180414/-0.0005/particles-surf-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180331_20180414/-0.0005/particles-surf-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180331_20180414/-0.0005/particles-surf-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180331_20180414/-0.0005/particles-surf-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180331_20180414/-0.005/particles-avg-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180331_20180414/-0.005/particles-avg-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180331_20180414/-0.005/particles-avg-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180331_20180414/-0.005/particles-avg-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180331_20180414/-0.005/particles-bed-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180331_20180414/-0.005/particles-bed-44days-v07ABnofiber-coast.png: processing
INFO:roo

INFO:root:processed/20180428_20180512/-0.0005/particles-surf-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180428_20180512/-0.0005/particles-surf-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180428_20180512/-0.0005/particles-surf-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180428_20180512/-0.005/particles-avg-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180428_20180512/-0.005/particles-avg-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180428_20180512/-0.005/particles-avg-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180428_20180512/-0.005/particles-avg-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180428_20180512/-0.005/particles-bed-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180428_20180512/-0.005/particles-bed-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180428_20180512/-0.005/particles-bed-44days-v07ABstd-bay.png: processing
INFO:root:proc

INFO:root:processed/20180529_20180612/-0.0005/particles-surf-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180529_20180612/-0.0005/particles-surf-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180529_20180612/-0.005/particles-avg-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180529_20180612/-0.005/particles-avg-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180529_20180612/-0.005/particles-avg-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180529_20180612/-0.005/particles-avg-44days-v07ABstd-coast.png: processing
INFO:root:processed/20180529_20180612/-0.005/particles-bed-44days-v07ABnofiber-bay.png: processing
INFO:root:processed/20180529_20180612/-0.005/particles-bed-44days-v07ABnofiber-coast.png: processing
INFO:root:processed/20180529_20180612/-0.005/particles-bed-44days-v07ABstd-bay.png: processing
INFO:root:processed/20180529_20180612/-0.005/particles-bed-44days-v07ABstd-coast.png: processing
INFO:root:processed/

In [ ]:
class MissingFiles(Exception): pass

class WsPanels(object):
    all_w_s=["-0.05","-0.005","-0.0005",
             "0.0",
             "0.0005","0.005","0.05"]
    figsize=(8.4,8)
    num=None
    PlotClass=BayConcFigure
    shape=(3,3)
    # weird python rules - tries to make this an instance method.
    panel_labeler=staticmethod(lambda i: "abcdefghijkl"[i])
    # for text label in each panel
    panel_txt_loc=[0.50,0.7]
    
    def loader(self,ptm_conc):
        return load_ptm_conc(ptm_conc)
    
    @classmethod
    def expand_ptm_conc(cls,ptm0_conc):
        ptm_concs=[ptm0_conc.replace("/0.0/","/"+w_s+"/")
                   for w_s in cls.all_w_s]
        if not all([os.path.exists(fn) for fn in ptm_concs]):
            log.warning("{ptm_conc} did not have all w_s - skip")
            raise MissingFiles()
        return ptm_concs
    
    def __init__(self,ptm0_conc,**kw):
        """
        ptm0_conc: path to the w_s=0.0 netcdf file.
        assumes that /0.0/ appears in the path, and
        can be replaced by the other settling velocities
        """
        utils.set_keywords(self,kw)
        
        ptm_concs=self.expand_ptm_conc(ptm0_conc)
        
        if self.num is not None:
            fig=plt.figure(figsize=self.figsize,num=self.num)
            fig.clf()
            fig.set_size_inches(self.figsize,forward=True)            
        self.fig,self.axs=plt.subplots(self.shape[0],self.shape[1],
                                       num=self.num,figsize=self.figsize)
        
        self.bfs=[]
        sum_conc=0.0
        for w_s,ptm_conc,ax in zip(self.all_w_s+['sum'],
                                   ptm_concs+['sum'],
                                   self.axs.ravel()):
            kw={}
            # Monkeys!
            kw['average_label']=lambda: ""
            kw['date_label']=lambda: ""
            if ptm_conc!=ptm_concs[0]:
                kw['cax_loc']=None
            else:
                # kw['cax_loc']=[0.68,0.03,0.02,0.28]
                kw['cax_loc']=[0.68,0.13,0.30,0.02]
                kw['cbar_args']={'orientation':'horizontal'}
            kw['txt_loc']=self.panel_txt_loc
            kw['fontsize']=12

            if ptm_conc=='sum':
                conc=sum_conc
                kw['behavior_label']=lambda: "Sum"
            else:
                ds,conc=self.loader(ptm_conc)
                sum_conc = sum_conc + conc
            bf=self.PlotClass(ds,conc=conc,ax=ax,fig=self.fig,**kw)
            self.bfs.append(bf)
            
        self.fig.subplots_adjust(hspace=0.0,wspace=0.0)
        ax=self.axs.ravel()[-1]
        plt.setp(ax.spines.values(),visible=0)
        ax.xaxis.set_visible(0)
        ax.yaxis.set_visible(0)
        
        sum_txts=[bf.date_label_ext(bf.ds),
                  bf.average_label_ext(bf.ds)]
        ax.text(0.10,0.90,"\n".join(sum_txts),transform=ax.transAxes,
               va='top',fontsize=12)
        # This is to avoid overwriting the neighboring spines.
        ax.patch.set_facecolor('none')
        
        if self.panel_labeler is not None:
            self.label_panels(self.axs.ravel()[:8])
    panel_label_loc=[0.05,0.9]
    panel_label_kw={'va':'top','fontsize':12,'fontweight':'bold'}
    def label_panels(self,axs):
        for i,ax in enumerate(axs):
            txt=self.panel_labeler(i)
            ax.text(self.panel_label_loc[0],self.panel_label_loc[1],txt,
                    transform=ax.transAxes,**self.panel_label_kw)
        
class WsPanelsCoastal(WsPanels):
    panel_label_loc=[0.02,0.16]
    panel_txt_loc=[0.02,0.08]
    PlotClass=CoastalConcFigure

In [ ]:
# 8-panel plots
# Bay and coast
# show each of the settling velocities, and a summation.

display=False
force=True

if not display: plt.ioff()

# starts with 15 day datasets -- pull passive, and generate the
# rest

# filter the original list down to just the passive
#fns=glob.glob('processed/201*_201*/0.0/particles*-vNN.nc')
#fns.sort()

for ptm_conc in single_fns():
    if "/0.0/" not in ptm_conc:
        continue    
    
    for config in ['bay','coast']:
        img_fn=ptm_conc.replace("0.0/","").replace('.nc',f'-{config}-8panel.png')
        if (not display) and os.path.exists(img_fn) and not force: 
            log.info(f"{img_fn}: exists")
            continue
        log.info(f"{img_fn}: processing")
        if config=='coast':
            pf=WsPanelsCoastal(ptm_conc)
        else:
            pf=WsPanels(ptm_conc)
        if display:
            break
        pf.fig.savefig(img_fn,dpi=150)
        plt.close(pf.fig)
    if display:
        break
if not display: plt.ion()

In [ ]:
# Combined, panels
# Maybe this is monkeyed sufficiently.
display=False

if not display:
    plt.ioff()
else:
    plt.ion()
for ptm_concA in combined_fns():
    if "/0.0/" not in ptm_concA:
        continue
        
    for config in ['coast']: # ['bay','coast']:
        img_fn=re.sub(r'(v\d\d)A',r'\1AB45',ptm_concA)
        img_fn=img_fn.replace("0.0/","").replace('.nc',f'-{config}-8panel.png')
    
        if (not display) and os.path.exists(img_fn): 
            log.info(f"{img_fn}: exists")
            continue
        log.info(f"{img_fn}: processing")
        #ds,conc=load_combined(ptm_concA)
        kw=dict(extra_text=['44 day cutoff'])
        
        if config=='coast':
            pf=WsPanelsCoastal(ptm_concA,loader=load_combined)
        else:
            pf=WsPanels(ptm_concA,loader=load_combined)
        if display:
            break
        pf.fig.savefig(img_fn,dpi=150)
        plt.close(pf.fig)
    if display:
        break
if not display:
    plt.ion()
    

In [ ]:
# Plot a sum, surface plot vs. the manta
manta=pd.read_csv('manta_summary.csv')
manta

In [ ]:
# So I want a sum concentration representative of wet weather.
# either 2/28 or 3/02.
# how the 2/28, 44 dayer.
display=True
force=True
max_age=15

for max_age in [15,44]:
    for season in ['Wet','Dry']:
        # find the base ptm_conc
        kw={}
        if max_age==44:
            fns=combined_fns()
            loader=load_combined
            kw['extra_text']=['44 day cutoff']
        elif max_age==15:
            fns=single_fns()
            loader=load_ptm_conc
        else:
            raise Exception(f"Wha kind of max age is {max_age}")

        for ptm_concA in fns:
            if "/0.0/" not in ptm_concA:
                continue
            if ( (season=='Wet') and 
                 ('20180228' not in ptm_concA) and
                 ('20180302' not in ptm_concA) ):
                continue
            if ( (season=='Dry') and 
                 ('20170828' not in ptm_concA) and
                 ('20170830' not in ptm_concA) ):
                continue
            if 'surf' not in ptm_concA:
                continue

            config='coast'
            img_fn=ptm_concA
            if max_age>15: # i.e. it has A and B parts
                img_fn=re.sub('(v\d\d)A',r'\1AB',ptm_concA) # .replace('vNNA','vNNAB45')
            img_fn=img_fn.replace("0.0/","").replace('.nc',f'-{config}-vs_manta-{max_age}days.png')

            if (not display) and (not force) and os.path.exists(img_fn): 
                log.info(f"{img_fn}: exists")
                continue
            break
        # Get the full list of all settling velocities
        ptm_concs=WsPanels.expand_ptm_conc(ptm_concA)
        # Sum to get one concentration.
        conc=0.0
        for ptm_conc in ptm_concs:
            ds,conc_one=loader(ptm_conc)
            conc=conc+conc_one

        fig,ax=plt.subplots(1,1,figsize=(8,5))

        F=CoastalConcFigure(ds,conc,vmin=1e-3,vmax=1e2,fig=fig,ax=ax,
                            behavior_label=lambda: "Sum",
                            cax_loc=[0.05,0.2,0.03,0.65])
        ax.axis('equal')
        # Plot the manta data.
        manta_sel=manta[ (manta.SampleType=='Trawl') & (manta.Season==season) ]
        bad=manta_sel['part_per_m2']<=0.0
        ax.scatter(manta_sel['x'][~bad],
                   manta_sel['y'][~bad],
                   70,
                   manta_sel['part_per_m2'][~bad],
                   cmap='CMRmap_r',
                   edgecolors='w',
                   norm=colors.LogNorm(vmin=1e-3,vmax=100),zorder=3)
        ax.plot(manta_sel['x'][bad],
                manta_sel['y'][bad],
                'kx')
        # Annotate what the typical minimum possible concentration is --
        # i.e. 1 particle in the whole sample volume.
        mdl=1./ np.median(manta.area_m2) # manta.volume_l/1000.
        F.cax.axhline(mdl,color='k')

        ax.axis( (430820., 614360., 4119898., 4234611.) )
        fig.savefig(img_fn,dpi=150)
        log.info(f"Saved image to {img_fn}")

In [ ]:
sed=pd.read_csv('sed_data_grouped.csv',index_col=0)
sed['name']=sed.index.values
sed

In [ ]:
re.sub(r'.*-(v.*)\.nc',r'\1',single_fns()[0])

In [ ]:
# average across all available.
display=False
force=True

for max_age in [44,15]:
    conc_accum=0.0 # accumulate concentrations for average.
    Naccum=0 # count how many fields have been added in here.

    t_min=None
    t_max=None

    # find the base ptm_conc
    if max_age==44:
        fns=combined_fns()
        loader=load_combined
    else:
        fns=single_fns()
        loader=load_ptm_conc

    for ptm_concA in fns:
        if "/0.0/" not in ptm_concA:
            continue
        if 'bed' not in ptm_concA:
            continue

        # Get the full list of all settling velocities
        ptm_concs=WsPanels.expand_ptm_conc(ptm_concA)
        # Sum to get one concentration.
        conc_sum=0.0
        for ptm_conc in ptm_concs:
            ds,conc_one=loader(ptm_conc)
            conc_sum=conc_sum+conc_one
            if t_min is None or t_min>ds.time_start:
                t_min=ds.time_start
            if t_max is None or t_max<ds.time_end:
                t_max=ds.time_end
        conc_accum=conc_accum+conc_sum
        Naccum+=1
        
        # grab the config info opportunistically
        model_data_label=re.sub(r'.*-(v.*)\.nc',r'\1',single_fns()[0])

    conc_mean = conc_accum / Naccum
    ds=ds.copy()
    ds['time_start']=(),t_min
    ds['time_end']=(),t_max

    raise Exception(f"Just checking - model data label is {model_data_label}")
    img_fn=f"processed/sediment-bed-{model_data_label}-{config}-{max_age}days.png"

    if (not display) and (not force) and os.path.exists(img_fn): 
        log.info(f"{img_fn}: exists")
        continue
    
    F=CoastalConcFigure(ds,conc_mean,vmin=1e-3,vmax=1e2,
                        behavior_label=lambda: "Sum over $w_s$, annual mean",
                        average_label=lambda: f"Near bed, max. age {max_age} days",
                        figsize=(7,6),
                        cax_loc=[0.05,0.2,0.03,0.65],
                        fontsize=12,
                        zoom=[485239., 602792., 4137255., 4238015.])

    if 1:
        # kludgy label postioning.
        sed['xoff']=0.0 ; sed['yoff']=0.0
        sed.loc['San Pablo Bay','yoff']=12000
        sed.loc['San Pablo Bay','xoff']=-6000
        sed.loc['Suisun Bay','yoff']=-10000
        sed.loc['Central Bay','xoff']=18000
        sed.loc['Tomales Bay','xoff']=6000
        sed.loc['South Bay','xoff']=21000
        sed.loc['Lower South Bay','xoff']=-18500
        sed.loc['Lower South Bay','yoff']=-600
        sed.loc['Southern Sloughs','xoff']=-18000
        sed.loc['Southern Sloughs','yoff']=-3000
        for idx,row in sed.iterrows():
            F.ax.text(row['x']+row['xoff'],row['y']+row['yoff'],
                     f"{row['name']}: {row['part_per_mass']:.2f}",
                     ha='center')
    F.ax.axis(F.zoom)
    if not display:
        F.fig.savefig(img_fn,dpi=150)
        log.info(f"Saved image to {img_fn}")